In [37]:
import requests
import json
from bs4 import BeautifulSoup as bs4
from tqdm import tqdm

url = "https://www.123pilzsuche-2.de/daten/details/2005Alphabethisch.htm"

In [38]:
res = requests.get(url)
soup = bs4(res.text)

In [39]:
results = []
trs = soup.findAll("table", {"class": "MsoNormalTable"})[0].findAll("tr")[1:]
for tr in trs:
  tds = tr.findAll("td")
  htm = tds[0].findAll("a")[0].attrs.get("href")
  url = f"https://www.123pilzsuche-2.de/daten/details/{htm}"
  if speisewert := tds[2].text.strip():
    results.append((url, speisewert))

In [50]:
import lxml
data = []
for result in tqdm(results):
  uri, speisewert = result
  res = requests.get(uri)
  soup = bs4(res.text, features="lxml")
  try:
    lat_name = soup.findAll("font", {"size": "1", "color": "black", "face": "Arial"})[0].text
  except IndexError:
    continue
  try:
    table = soup.findAll("table", {"class": "MsoNormalTable"})[1]
  except IndexError:
    table = soup.findAll("table", {"class": "MsoNormalTable"})[0]
  trs = table.findAll("tr")[1:]
  t = {
    "lateinischer name": lat_name,
    "speisewert": speisewert
  }
  try:
    name = soup.findAll("font", {"size": "4"})[0].text.strip()
  except Exception:
    name = "Unbekannt"
  t["name"] = name
  for tr in trs:
    tds = tr.findAll("td")
    if (key := tds[0].text.replace(":", "").strip()) in ["Geruch", "Geschmack", "Fleisch", "Stiel", "Vorkommen", "Hut", "Lamellen"]:
      t[key] = tds[1].text.strip()
  data.append(t)

with open("../data/pilze.json", "w") as f:
  json.dump(data, f, indent=4)

  0%|          | 1/4061 [00:00<38:27,  1.76it/s]/usr/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
100%|██████████| 4061/4061 [58:55<00:00,  1.15it/s]  


In [45]:
data

[{'lateinischer name': 'ORBILIA EUONYMI',
  'speisewert': 'F - ungenießbar',
  'Geruch': 'Neutral.',
  'Geschmack': 'Unbedeutend.',
  'Hut': '0,5-2 (3) mm Ø, orange, orangerot, gelblich, bernsteingelb, trocken orangegelb, jung\n  halbkugelig, später schüsselförmig. Oberfläche etwas körnig, ohne Haare.',
  'Fleisch': 'Gelblich bis orangegelb.',
  'Stiel': 'Fehlt, auf dem Substrat aufliegend.',
  'Vorkommen': 'An totem faulendem Laubholz, Folgezersetzer, ganzjährig, Hauptzeit:\n  Sommer bis Herbst.'},
 {'lateinischer name': 'DIDYMIUM DIFFORME (SYN.\nDIDERMA DIFFORME, PHYSARUM DIFFORME, DIDERMA NITENS, CHONDRIODERMA DIFFORME)',
  'speisewert': 'F - ungenießbar',
  'Geruch': 'Neutral.',
  'Geschmack': 'Unbedeutend.',
  'Fleisch': 'Hellbraun, grünbraun, olivbraun, schwarzbraun, schwarz.',
  'Stiel': 'Fehlt, flach auf dem Substrat aufliegend.',
  'Vorkommen': 'Auf Moosen oder verschiedenen Laub- und Nadelhölzern, Folgezersetzer:\n  +/- Saprophyt; +/- Parasitär = Phagocytose,\n  Schleimpilze 